<a href="https://colab.research.google.com/github/toche7/DataAnalytic/blob/main/Lab05Multicollinearity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multicollinearity, Residuals และการนำเสนอผลลัพธ์

ใน Lab นี้ เราจะมาเรียนรู้เกี่ยวกับการตรวจสอบ Multicollinearity, การวิเคราะห์ Residuals และวิธีการนำเสนอผลลัพธ์ของโมเดล Linear Regression

---

## 1. Import Libraries & Load Data

### วัตถุประสงค์:
นำเข้าไลบรารีที่จำเป็นและโหลดข้อมูลที่เราได้เตรียมไว้แล้ว

### คำแนะนำ:
*   นำเข้าไลบรารี `pandas`, `numpy`, `statsmodels.api`, `matplotlib.pyplot`, `seaborn`.
*   โหลด DataFrame ที่ผ่านการจัดการ Missing/Outlier และได้ตรวจสอบ homogeneity & correlation แล้ว (สมมติว่ามีไฟล์ `cleaned_data.csv` อยู่)

In [ ]:
# 1. Import Libraries & Load Data

# Import Libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
f1 = np.random.rand(100)
data = {
    'Feature1': f1,
    'Feature2': f1   + 0.5 * np.random.rand(100), # มีความสัมพันธ์กับ Feature1
    'Feature3': f1  + 0.3 * np.random.rand(100), # มีความสัมพันธ์กับ Feature1 บ้าง
    'Feature4': np.random.rand(100), # ตัวแปรที่อาจไม่มีความสัมพันธ์
    'Target': np.random.rand(100) * 3 + (np.random.rand(100) * 2) + (np.random.rand(100) * 1.5) + np.random.rand(100)
}
df = pd.DataFrame(data)
print("Using dummy data with some correlation for demonstration.")
print(df.head())

## 2. ตรวจสอบ Multicollinearity ด้วย VIF

### วัตถุประสงค์:
ตรวจสอบปัญหา Multicollinearity ในตัวแปรอิสระโดยใช้ Variance Inflation Factor (VIF)

### คำแนะนำ:
*   เตรียม matrix `X` ของตัวแปรอิสระที่คุณจะใช้ในโมเดล
*   คำนวณค่า VIF สำหรับแต่ละตัวแปรใน `X` โดยใช้ฟังก์ชัน `variance_inflation_factor` จาก `statsmodels.stats.outliers_influence`.
*   แสดงผลค่า VIF ในตารางและตีความ:
    *   ค่า VIF ที่สูง (มักใช้เกณฑ์ > 5 หรือ > 10) บ่งชี้ถึงปัญหา Multicollinearity ที่รุนแรง
    *   ควรพิจารณาตัดตัวแปรที่มี VIF สูงออกจากโมเดล

In [ ]:
# 2. ตรวจสอบ Multicollinearity ด้วย VIF

# Prepare independent variables matrix
# *** ระบุชื่อคอลัมน์ของตัวแปรอิสระ (Independent Variables) ที่คุณต้องการตรวจสอบ Multicollinearity ***
# *** แทนที่รายการด้านล่างด้วยชื่อคอลัมน์จริงของคุณ ***
independent_variables_to_check_vif = ['Feature1', 'Feature2', 'Feature3', 'Feature4']

X_vif = df[independent_variables_to_check_vif]

# Add constant (intercept) for VIF calculation in statsmodels
X_vif = sm.add_constant(X_vif)


# Interpretation:
# VIF = 1: No correlation with other variables
# 1 < VIF < 5: Low to moderate correlation
# VIF > 5 or 10: High multicollinearity, consider handling

In [ ]:
X_vif

In [ ]:
X_vif.values

In [ ]:

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i)
                   for i in range(len(X_vif.columns))]

# Display and interpret results
print("VIF Results:")
print(vif_data)

## 3. จัดการ Multicollinearity

### วัตถุประสงค์:
จัดการกับปัญหา Multicollinearity ที่ตรวจพบในขั้นตอนก่อนหน้า

### คำแนะนำ:
*   จากผลการตรวจสอบ VIF ให้พิจารณาเลือกตัดตัวแปรที่มีค่า VIF สูงออก
*   หรืออาจพิจารณาการรวมกลุ่มตัวแปรที่มีความสัมพันธ์กันสูง


In [ ]:
# prompt: remove feature 1

# 3. จัดการ Multicollinearity

# Prepare independent variables matrix AFTER handling multicollinearity
# *** จากผล VIF ให้เลือกตัวแปรอิสระที่ต้องการใช้ในโมเดลสุดท้าย ***
# *** แทนที่รายการด้านล่างด้วยชื่อคอลัมน์จริงของคุณ ***
independent_variables_final = ['Feature2', 'Feature3', 'Feature4'] # Example: Removing Feature1

X = df[independent_variables_final]
y = df['Target']

# Add a constant (intercept) to the independent variables matrix for the regression model
X = sm.add_constant(X)

print("\nIndependent variables used in the final model:")
print(X.head())

In [ ]:
# prompt: check VIF again

import pandas as pd
# Re-calculate VIF for the final set of independent variables
X_vif_final = X.drop(columns=['const']) # Drop the constant column for VIF calculation

# Add constant back for VIF calculation
X_vif_final = sm.add_constant(X_vif_final)


# Calculate VIF for each variable in the final set
vif_data_final = pd.DataFrame()
vif_data_final["feature"] = X_vif_final.columns
vif_data_final["VIF"] = [variance_inflation_factor(X_vif_final.values, i)
                         for i in range(len(X_vif_final.columns))]

# Display and interpret results for the final model variables
print("\nRe-calculated VIF Results for Final Model Variables:")
vif_data_final

# Interpretation:
# VIF = 1: No correlation with other variables
# 1 < VIF < 5: Low to moderate correlation
# VIF > 5 or 10: High multicollinearity, consider handling (if not already done)

## 4. Fit Linear Regression Model & วิเคราะห์ Residuals

### วัตถุประสงค์:
สร้างโมเดล Linear Regression และวิเคราะห์ Residuals เพื่อตรวจสอบข้อสมมติฐานของ OLS

### คำแนะนำ:
*   ใช้ไลบรารี `statsmodels.api` เพื่อสร้างและ fit โมเดล Linear Regression (OLS - Ordinary Least Squares) โดยใช้ตัวแปรอิสระที่ผ่านการจัดการ Multicollinearity แล้ว
*   แสดงตารางสรุปผลลัพธ์ของโมเดล (`model.summary()`) เพื่อดูค่าสัมประสิทธิ์ (coef), p-value, ค่า R-squared และข้อมูลทางสถิติอื่นๆ
*   ดึงค่า Residuals (`model.resid`) และ Fitted Values (`model.fittedvalues`) จากโมเดลที่ fit แล้ว

In [ ]:
# 4. Fit Linear Regression Model & วิเคราะห์ Residuals

# *** Specify the name of the dependent variable (Target Variable) ***
# *** Replace 'Target' with your actual column name ***
dependent_variable_for_model = 'Target'

# Prepare data for the model using variables after multicollinearity handling
X_model = df[independent_variables_final]
y_model = df[dependent_variable_for_model]

# Add constant (intercept) to independent variables for OLS model
X_model = sm.add_constant(X_model)

# Create and Fit the Linear Regression Model (OLS)
model = sm.OLS(y_model, X_model).fit()

# Print the model summary
print("\nModel Summary:")
print(model.summary())

# Extract residuals and fitted values
residuals = model.resid
fitted_values = model.fittedvalues

print("\nResiduals and Fitted Values extracted from the model.")

## 5. Visualize Residuals

### วัตถุประสงค์:
แสดงภาพ Residuals เพื่อตรวจสอบการกระจายตัว (Normality) และรูปแบบ (Pattern) ของ Residuals

### คำแนะนำ:
*   **Histogram / KDE Plot**: พล็อต histogram หรือ Kernel Density Estimate (KDE) ของ Residuals เพื่อดูว่ามีการกระจายตัวแบบปกติ (Normal Distribution) หรือไม่
*   **QQ-plot**: สร้าง Quantile-Quantile Plot (QQ-plot) ของ Residuals เพื่อเปรียบเทียบการกระจายตัวของ Residuals กับการกระจายตัวแบบปกติ หากจุดอยู่บนเส้นตรง แสดงว่า Residuals มีการกระจายตัวใกล้เคียงปกติ
*   **Scatter Plot (Fitted vs. Residuals)**: พล็อต scatter plot ระหว่าง Fitted Values บนแกน X และ Residuals บนแกน Y เพื่อตรวจสอบว่ามีรูปแบบ (pattern) ใดๆ หรือไม่ หากไม่มีรูปแบบที่ชัดเจนและ Residuals กระจายตัวรอบๆ แกน Y = 0 แสดงว่าข้อสมมติฐานเรื่อง Homoscedasticity (ความแปรปรวนของ Residuals คงที่) อาจเป็นจริง

In [ ]:
# 5. Visualize Residuals

# Set seaborn style for plots
sns.set_style("whitegrid")

# Histogram / KDE of residuals
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(residuals, kde=True)
plt.title("Histogram of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")

# QQ-plot to check normality
plt.subplot(1, 2, 2)
sm.qqplot(residuals, line='s') # line='s' plots a standard normal line
plt.title("QQ-plot of Residuals")
plt.show()

# Scatter plot (fitted vs. residuals) to check for patterns
plt.figure(figsize=(8, 5))
plt.scatter(fitted_values, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--') # Add a horizontal line at y=0
plt.title("Residuals vs. Fitted Values")
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.show()

print("Residual plots generated.")

## 6. Prepare Presentation Charts

### วัตถุประสงค์:
สร้างกราฟเพื่อนำเสนอผลลัพธ์ของโมเดลและการวิเคราะห์

### คำแนะนำ:
*   **Ranking Chart**: จัดเรียงลำดับความสำคัญของตัวแปร (เช่น ตามค่าสัมประสิทธิ์หรือ p-value) และนำเสนอด้วย Bar Chart
*   **Comparison Chart**: ใช้ Grouped Bar Chart หรือ Stacked Bar Chart เพื่อเปรียบเทียบค่าหรือผลกระทบระหว่างกลุ่มต่างๆ
*   **Trend Chart**: หากมีการวิเคราะห์ข้อมูลอนุกรมเวลาหรือมีตัวแปรที่มีลำดับ ให้ใช้ Line Plot เพื่อแสดงแนวโน้มการเปลี่ยนแปลง
*   **Frequency Table**: ใช้ `value_counts()` เพื่อนับความถี่ของค่าในตัวแปร categorical และนำเสนอด้วย Bar Chart

In [ ]:
# 6. Prepare Presentation Charts

# Example of creating presentation charts (please adjust as needed)

# Ranking: Bar chart of Coefficients
# Extract coefficients from the model
coefficients = model.params.drop('const') # Exclude intercept
coefficients = coefficients.sort_values(ascending=False) # Sort

plt.figure(figsize=(10, 6))
coefficients.plot(kind='bar')
plt.title("Ranking of Variable Coefficients")
plt.ylabel("Coefficient Value")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Example: Comparison - Grouped Bar Chart (Assuming you have a categorical variable 'Category' and want to compare the mean of 'Target' by group)
# if 'Category' in df.columns:
#     # Calculate mean of target per category
#     mean_target_by_category = df.groupby('Category')[dependent_variable_for_model].mean()
#     plt.figure(figsize=(8, 5))
#     mean_target_by_category.plot(kind='bar')
#     plt.title(f"Average {dependent_variable_for_model} by Category")
#     plt.xlabel("Category")
#     plt.ylabel(f"Average {dependent_variable_for_model}")
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()
#     plt.show()

# Example: Frequency Table + Bar Chart for a Categorical Variable (if applicable)
# if 'Category' in df.columns:
#     category_counts = df['Category'].value_counts()
#     plt.figure(figsize=(8, 5))
#     category_counts.plot(kind='bar')
#     plt.title("Frequency of Categories")
#     plt.xlabel("Category")
#     plt.ylabel("Count")
#     plt.xticks(rotation=45, ha='right')
#     plt.tight_layout()
#     plt.show()

print("Example presentation charts generated.")

### สรุป
ความสำคัญของ Multicollinearity, Residuals และการนำเสนอผลลัพธ์

- **ป้องกันปัญหา Multicollinearity**  
  - ช่วยระบุและลดความซ้ำซ้อนของตัวแปรอิสระก่อนสร้างโมเดล  
  - ปรับปรุงความเสถียรของค่าสัมประสิทธิ์ (coefficients) และค่าคาดการณ์  

- **ยืนยันสมมติฐานของ Linear Regression**  
  - ตรวจสอบ residuals เพื่อดูว่าการแจกแจงใกล้เคียงปกติ (normality) และไม่มี pattern ที่บ่งชี้ปัญหา  
  - ช่วยให้มั่นใจว่าโมเดล OLS มีความถูกต้องและผลลัพธ์เชื่อถือได้  

- **ปรับปรุงการเลือกตัวแปร (Feature Selection)**  
  - ใช้ VIF ในการตัดตัวแปรที่มี multicollinearity สูง หรือพิจารณาใช้เทคนิคเชิงมิติ (PCA, Regularization)  
  - ส่งเสริมการสร้างโมเดลที่กระชับและมีประสิทธิภาพ  

- **เตรียมการนำเสนอผลลัพธ์อย่างมืออาชีพ**  
  - สร้างกราฟและตาราง (ranking, comparison, trend, frequency) เพื่อสื่อสารข้อค้นพบได้ชัดเจน  
  - ช่วยให้ผู้ฟังเข้าใจจุดเด่น–จุดด้อยของโมเดล และข้อเสนอแนะเบื้องต้น  

- **เชื่อมต่อกับขั้นตอน Modeling ถัดไป**  
  - เป็นจุดตรวจคุณภาพข้อมูลก่อนใช้งานจริงในขั้นตอน predictive modeling  
  - ลดความเสี่ยงของการ overfit และเพิ่มความ generalizability ของโมเดล  